# **Config:** 

In [ ]:
# NOTE: paths are Kaggle-style; change BASE/INPUT if running locally.

import os
from pathlib import Path

# ==== detect environment ====
IS_KAGGLE = os.path.exists("/kaggle")

# ==== base dirs ====
BASE = Path("/kaggle/working") if IS_KAGGLE else Path("./working")     
INPUT = Path("/kaggle/input") if IS_KAGGLE else Path("./input")        

# ==== repo / checkpoints / exports ====
REPO = BASE / "cyclegan_repo"
CKPTDIR = BASE / "checkpoints_local"
RUN_DIR = CKPTDIR / "my_cyclegan"
EXPORT_DIR = BASE / "export_my_cyclegan_latest"
RESULTS = BASE / "results"
LOG = BASE / "train_live.log"

# ==== kaggle dataset (for pushing checkpoints) ====
OWNER = "shiristern"                             # user/owner name in kaggle
SLUG = "checkpoints"                             # dataset name for saving data
ID = f"{OWNER}/{SLUG}"
SRC = INPUT / "checkpoints"                      # where to copy checkpoints FROM

# ==== main CycleGAN dataset (A/B) ====
DATAROOT = INPUT / "cyclegan-mydata" / "content" / "drive" / "MyDrive" / "cyclegan_datasets" / "mydata"
TEST_A = DATAROOT / "testA"                      # you mentioned this
TEST_B = DATAROOT / "testB"                      # and this
REAL_A = TEST_A                                  # for FID/KID, real A
REAL_B = TEST_B                                  # for FID/KID, real B

# ==== run params ====
NAME = "my_cyclegan"                             # must match folder under checkpoints_local
EPOCH = "250"                                    # or "latest" / "100" ...

# ==== generated images (CycleGAN standard output) ====
# example: /kaggle/working/results/my_cyclegan/test_250/images
GEN_DIR = RESULTS / NAME / f"test_{EPOCH}" / "images"

# ==== single-image test ====
IN_IMG = INPUT / "test-image-cyclegan" / "image9.png"             # input image for single test
WORK_IMG = BASE / "single" / "image9.png"                         # copy here for debugging
ROOT_SINGLE = BASE / "_single_BtoA"                               # minimal dataroot for single run
SINGLE_RESULTS = BASE / "single_results"                          # where single-image outputs go
DIR = SINGLE_RESULTS / NAME / f"test_{EPOCH}" / "images"          # <- you asked about this

# ==== FID/KID target folders ====
OUT_FAKE_B = BASE / "fid_AtoB_fakeB"                              # A->B: outputs of type fake_B
OUT_FAKE_A = BASE / "fid_BtoA_fakeA"                              # B->A: outputs of type fake_A

# ==== checkpoints produced elsewhere (your extra paths) ====
# G(A) -> looks like B
G_A = INPUT / "my-checkpoints" / f"epoch_{EPOCH}" / "fake_B"
# F(B) -> looks like A
F_B = INPUT / "my-checkpoints" / f"epoch_{EPOCH}" / "fake_A"

# ==== binary-classifier / real-vs-fake dataset root ====
ROOT = BASE / "real_vs_fake_jpeg256"                 # this was your ROOT = Path(...)
BEST_PATH = BASE / "best_resnet18_real_vs_fake.pt"

# ====== export to env so %%bash cells can use it ======
to_export = {
    "BASE": BASE,
    "INPUT": INPUT,
    "REPO": REPO,
    "CKPTDIR": CKPTDIR,
    "RUN_DIR": RUN_DIR,
    "EXPORT_DIR": EXPORT_DIR,
    "RESULTS": RESULTS,
    "LOG": LOG,
    "OWNER": OWNER,
    "SLUG": SLUG,
    "ID": ID,
    "SRC": SRC,
    "DATAROOT": DATAROOT,
    "TEST_A": TEST_A,
    "TEST_B": TEST_B,
    "REAL_A": REAL_A,
    "REAL_B": REAL_B,
    "NAME": NAME,
    "EPOCH": EPOCH,
    "GEN_DIR": GEN_DIR,
    "IN_IMG": IN_IMG,
    "WORK_IMG": WORK_IMG,
    "ROOT_SINGLE": ROOT_SINGLE,
    "SINGLE_RESULTS": SINGLE_RESULTS,
    "DIR": DIR,
    "OUT_FAKE_B": OUT_FAKE_B,
    "OUT_FAKE_A": OUT_FAKE_A,
    "G_A": G_A,
    "F_B": F_B,
    "REAL_VS_FAKE_ROOT": REAL_VS_FAKE_ROOT,
    "BEST_PATH": BEST_PATH,
}

for k, v in to_export.items():
    os.environ[k] = str(v)

print("✅ CONFIG loaded.")
print("BASE     :", os.environ["BASE"])
print("DATAROOT :", os.environ["DATAROOT"])
print("GEN_DIR  :", os.environ["GEN_DIR"])
print("TEST_A   :", os.environ["TEST_A"])


Checking:

In [ ]:
import os
keys = [
    "BASE", "INPUT", "DATAROOT",
    "GEN_DIR", "OUT_FAKE_A", "OUT_FAKE_B",
    "TEST_A", "TEST_B",
    "G_A", "F_B",
    "SINGLE_RESULTS", "DIR",
]
for k in keys:
    print(k, "→", os.environ.get(k, "<MISSING>"))

Check versions:

In [ ]:
import torch, torchvision, platform
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("Python:", platform.python_version())
print("Platform:", platform.platform())

Stop running processes:

In [ ]:
%%bash
pkill -f "/cyclegan_repo/train.py" 2>/dev/null || true
pkill -f "kaggle datasets version" 2>/dev/null || true

# **Preparations:**

Copy the checkpoints folder from input to WORKING:

In [ ]:
%%bash
set -e                         

mkdir -p "$RUN_DIR"
cp -v "$SRC"/latest_net_*.pth "$RUN_DIR"/ 2>/dev/null || true
cp -v "$SRC"/*_net_*.pth     "$RUN_DIR"/ 2>/dev/null || true
cp -v "$SRC"/{loss_log.txt,train_opt.txt,iter.txt,README.txt} "$RUN_DIR"/ 2>/dev/null || true

echo "Now in $RUN_DIR:"
ls -lh "$RUN_DIR" | head -n 50

Set the next epoch:

In [ ]:
%%bash
LAST=$(ls "$RUN_DIR"/*_net_G_A.pth 2>/dev/null | sed -n 's#.*/\([0-9]\+\)_net_G_A\.pth#\1#p' | sort -n | tail -n1)
if [ -z "${LAST:-}" ]; then NEXT=1; else NEXT=$((LAST+1)); fi
echo "LAST epoch = ${LAST:-none}"
echo "NEXT epoch_count = $NEXT" | tee ${BASE}/NEXT.txt

Check if a repository exists:

In [ ]:
%%bash
ls -ld ${BASE}/cyclegan_repo || echo "MISSING: ${BASE}/cyclegan_repo"
echo "searching for train.py under ${BASE}:"
find ${BASE} -maxdepth 3 -type f -name train.py 2>/dev/null | sed 's#^#/##'

Reclone the repository:

In [ ]:
%%bash
set -e
[ -d "$REPO" ] || git clone --depth 1 https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git "$REPO"
ls -lh "$REPO" | head -n 20

Check if metadata exists:

In [ ]:
%%bash
mkdir -p "$EXPORT_DIR"
[ -f "$EXPORT_DIR/dataset-metadata.json" ] && echo "HAS_METADATA=YES" || echo "HAS_METADATA=NO"

Create metadata if it does not exist:

In [ ]:
%%bash
set -e
cat > "$EXPORT_DIR/dataset-metadata.json" <<'JSON'
{
  "title": "CycleGAN checkpoints",
  "id": "shiristern/checkpoints",
  "licenses": [{"name": "CC0-1.0"}]
}
JSON
echo "metadata ready"

Copy the token:

In [ ]:
import os, shutil
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
shutil.copy("${INPUT}/kaggle-json/kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)
print("Token installed.")

Verify that the token was copied correctly:

In [ ]:
%%bash
ls -l ~/.kaggle/kaggle.json || echo "no token"

# Clear memory space:

Clean the export folder:

Delete unnecessary checkpoints:

In [ ]:
%%bash
set -e
if [ -d "$EXPORT_DIR" ]; then
  echo "Cleaning $EXPORT_DIR (keeping dataset-metadata.json only)…"
  find "$EXPORT_DIR" -mindepth 1 -maxdepth 1 ! -name 'dataset-metadata.json' -print -exec rm -rf {} +
fi
df -h ${BASE}

In [ ]:
%%bash
set -e
echo "Deleting iter_* checkpoints in $RUN_DIR …"
find "$RUN_DIR" -type f -name 'iter_*_net_*.pth' -print -delete
df -h ${BASE}

Perform additional unnecessary deletions:

In [ ]:
import re, pathlib
run_dir = pathlib.Path(RUN_DIR)
keep_last_epochs = 2   

if run_dir.is_dir():
    keep = {p.name for p in run_dir.glob("latest_net_*.pth")}
    epoch_files = {}
    for p in run_dir.glob("*_net_*.pth"):
        m = re.match(r"(\d+)_net_.*\.pth$", p.name)
        if m: epoch_files.setdefault(int(m.group(1)), []).append(p)
    epochs = sorted(epoch_files.keys())
    to_del = []
    if len(epochs) > keep_last_epochs:
        for e in epochs[:-keep_last_epochs]:
            for p in epoch_files[e]:
                if p.name not in keep: to_del.append(p)
    for p in to_del:
        try: p.unlink(); print("deleted:", p.name)
        except Exception as e: print("skip:", p, e)
else:
    print("run_dir not found:", run_dir)

Check memory status:

In [ ]:
%%bash
df -h ${BASE}
du -h -d1 ${BASE} | sort -h | tail -n 15

# Train the model:

In [ ]:
%%bash
set -euo pipefail

UPLOAD_EVERY_SECONDS=180                            # Upload a new version every few seconds

# 0) Clean old processes
pkill -f "/cyclegan_repo/train.py" 2>/dev/null || true
pkill -f "kaggle datasets version" 2>/dev/null || true
pkill -f "tail -F $LOG"            2>/dev/null || true
rm -f /tmp/train.pid /tmp/uploader.pid || true
: > "$LOG"

# 1) Dependency + Code
pip -q install dominate
[ -d "$REPO" ] || git clone --depth 1 https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git "$REPO"

# 2) Next epoch_count 
LAST=$(ls "$RUN_DIR"/*_net_G_A.pth 2>/dev/null | sed -n 's#.*/\([0-9]\+\)_net_G_A\.pth#\1#p' | sort -n | tail -n1)
if [ -z "${LAST:-}" ]; then NEXT=1; else NEXT=$((LAST+1)); fi
echo "→ epoch_count=${NEXT} (last=${LAST:-none})" | tee -a "$LOG"

# 3) Upload one version (if there is anything to upload)
upload_once() {
  mkdir -p "$EXPORT_DIR"
  # Do not delete dataset-metadata.json if it exists
  find "$EXPORT_DIR" -maxdepth 1 -type f ! -name 'dataset-metadata.json' -delete || true
  cp -a "$RUN_DIR"/latest_net_*.pth "$EXPORT_DIR"/ 2>/dev/null || true
  cp -a "$RUN_DIR"/*_net_*.pth    "$EXPORT_DIR"/ 2>/dev/null || true
  cp -a "$RUN_DIR"/train_opt.txt  "$EXPORT_DIR"/ 2>/dev/null || true
  cp -a "$RUN_DIR"/iter.txt       "$EXPORT_DIR"/ 2>/dev/null || true
  cp -a "$RUN_DIR"/loss_log.txt   "$EXPORT_DIR"/ 2>/dev/null || true
  [ -f "$EXPORT_DIR/README.txt" ] || echo "CycleGAN checkpoints" > "$EXPORT_DIR/README.txt"
  # If dataset-metadata.json does not exist yet, skip (the dataset creation cell will generate it)
  [ -f "$EXPORT_DIR/dataset-metadata.json" ] || return 0
  [ -n "$(ls -A "$EXPORT_DIR" | grep -v '^dataset-metadata.json$' || true)" ] || return 0
  kaggle datasets version -p "$EXPORT_DIR" -m "auto $(date +%F_%T)" -r tar || true
}

# 4) Loop to upload versions continuously
uploader_loop() {
  while true; do
    upload_once
    sleep "$UPLOAD_EVERY_SECONDS"
  done
}

# 5) Run training in the background with continuous log flushing
( stdbuf -oL -eL env PYTHONUNBUFFERED=1 \
  python -u "$REPO/train.py" \
    --dataroot "$DATAROOT" \
    --name my_cyclegan \
    --checkpoints_dir "$CKPTDIR" \
    --model cycle_gan \
    --continue_train \
    --epoch_count "$NEXT" \
    --n_epochs 100 --n_epochs_decay 100 \
    --batch_size 1 \
    --preprocess resize_and_crop --load_size 286 --crop_size 256 \
    --lambda_identity 0.5 \
    --save_epoch_freq 5 --save_by_iter --save_latest_freq 3000 \
    --print_freq 25 --num_threads 0 --no_html --verbose \
  2>&1 ) >> "$LOG" & echo $! > /tmp/train.pid

# 6) Upload versions in the background + live tail of the output
uploader_loop & echo $! > /tmp/uploader.pid

finish() {
  echo "[uploader] final upload & cleanup" | tee -a "$LOG"
  upload_once
  if [ -f /tmp/uploader.pid ]; then kill "$(cat /tmp/uploader.pid)" 2>/dev/null || true; rm -f /tmp/uploader.pid; fi
}
trap finish EXIT

echo "🏃 Training is running (PID $(cat /tmp/train.pid)). Showing live output:" | tee -a "$LOG"
exec tail -F "$LOG"

# Test images check:

In [ ]:
%%bash
# Run test images
set -e

# 0) Ensure the repo is in the location the script expects
if [ ! -d ${BASE}/cyclegan_repo ]; then
  if [ -d ${BASE}/pytorch-CycleGAN-and-pix2pix ]; then
    ln -s ${BASE}/pytorch-CycleGAN-and-pix2pix ${BASE}/cyclegan_repo
  else
    git clone --depth 1 https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git ${BASE}/cyclegan_repo
  fi
fi

# 1) Verify that test.py actually exists
[ -f "$REPO/test.py" ] || { echo "ERROR: missing $REPO/test.py"; echo "Found instead:"; find ${BASE} -maxdepth 3 -name test.py; exit 1; }

# 3) Minimal dependencies for test
pip -q install dominate pillow numpy >/dev/null

# 4) Run test A→B
python -u "$REPO/test.py" \
  --dataroot "$DATAROOT" \
  --name "$NAME" \
  --checkpoints_dir "$CKPTDIR" \
  --model cycle_gan \
  --phase test \
  --epoch "$EPOCH" \
  --direction AtoB \
  --num_test 100000 \
  --results_dir "$RESULTS" \
  --eval

# 5) Run test B→A
python -u "$REPO/test.py" \
  --dataroot "$DATAROOT" \
  --name "$NAME" \
  --checkpoints_dir "$CKPTDIR" \
  --model cycle_gan \
  --phase test \
  --epoch "$EPOCH" \
  --direction BtoA \
  --num_test 100000 \
  --results_dir "$RESULTS" \
  --eval

echo "Saved under: ${RESULTS}/${NAME}/test_${EPOCH}/images"
ls -lh "${RESULTS}/${NAME}/test_${EPOCH}/images" | head -n 30 || true

Zip the results folder:

In [ ]:
%%bash
cd ${BASE}/checkpoints_local
zip -r test_latest_images.zip my_cyclegan

Run the model on a specific image:

In [ ]:
%%bash
set -e

# 0) Verify repo and basic dependencies
[ -d "$REPO" ] || git clone --depth 1 https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git "$REPO" >/dev/null
pip -q install dominate pillow numpy >/dev/null

# 1) Copy the image to working (for easier debugging)
mkdir -p "$(dirname "$WORK_IMG")"
cp -v "$IN_IMG" "$WORK_IMG"

# 2) Build a minimal dataroot with two folders (required by UnalignedDataset)
rm -rf "$ROOT_SINLGE"
mkdir -p "$ROOT_SINLGE/testA" "$ROOT_SINLGE/testB"

# Direction B→A: place the image in testB; put a dummy file in testA so it’s not empty
cp -v "$WORK_IMG" "$ROOT_SINLGE/testB/one.jpg"
cp -v "$WORK_IMG" "$ROOT_SINLGE/testA/dummy.jpg"

# 3) Run test.py
python -u "$REPO/test.py" \
  --dataroot "$ROOT_SINLGE" \
  --name "$NAME" \
  --checkpoints_dir "$CKPTDIR" \
  --model cycle_gan \
  --phase test \
  --epoch "$EPOCH" \
  --direction BtoA \
  --num_test 1 \
  --results_dir "$RESULTS" \
  --eval --serial_batches

# 4) Show where the results were saved and how many files were generated
OUTDIR="${SINGLE_RESULTS}/${NAME}/test_${EPOCH}/images"
echo "✅ Saved to: ${OUTDIR}"
ls -lh "${OUTDIR}" | head -n 30
echo "Total files:" $(ls -1 "${OUTDIR}" | wc -l)

Display the generated image next to the input image:

In [ ]:
from pathlib import Path
import math
from PIL import Image
import matplotlib.pyplot as plt

# Config
EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tif", ".tiff", ".webp"}  
PAGE = 1          
PER_PAGE = 48     
COLS = 6          
CELL_SIZE = 3     

# Collect files
paths = [p for p in Path(DIR).rglob("*") if p.suffix.lower() in EXTS]
paths.sort()
total = len(paths)
if total == 0:
    print(f"No images found in path: {DIR}")
else:
    start = (PAGE - 1) * PER_PAGE
    end = min(start + PER_PAGE, total)
    subset = paths[start:end]
    rows = math.ceil(len(subset) / COLS) or 1

    # Plotting
    plt.figure(figsize=(COLS * CELL_SIZE, rows * CELL_SIZE))
    for i, p in enumerate(subset, 1):
        try:
            img = Image.open(p)
            plt.subplot(rows, COLS, i)
            plt.imshow(img)
            plt.axis("off")
            plt.title(p.name, fontsize=8)
        except Exception as e:
            print(f"Skipping {p}: {e}")

    plt.tight_layout()
    plt.show()
   print(f"Showing {len(subset)} of {total} images from folder: {DIR} | Page {PAGE} ({start+1}–{end})")

# Evaluate metrics:

Copy images to the desired folder for metrics evaluation:

In [ ]:
%%bash
set -euo pipefail

# Preparation
if [ ! -d "$GEN_DIR" ]; then
      echo "❌ No results directory found: $GEN_DIR"
  exit 1
fi

echo "Cleaning and creating target folders…"
rm -rf "$OUT_FAKE_B" "$OUT_FAKE_A"
mkdir -p "$OUT_FAKE_B" "$OUT_FAKE_A"

# Copy: fake_B and fake_A
echo "Copying fake_B → $OUT_FAKE_B"
find "$GEN_DIR" -maxdepth 1 -type f \( -iname "*_fake_B.png" -o -iname "*_fake_B.jpg" -o -iname "*_fake_B.jpeg" \) -exec cp -t "$OUT_FAKE_B" {} +
echo "Copying fake_A → $OUT_FAKE_A"
find "$GEN_DIR" -maxdepth 1 -type f \( -iname "*_fake_A.png" -o -iname "*_fake_A.jpg" -o -iname "*_fake_A.jpeg" \) -exec cp -t "$OUT_FAKE_A" {} +

# Short report
cB=$(find "$OUT_FAKE_B" -type f | wc -l)
cA=$(find "$OUT_FAKE_A" -type f | wc -l)
echo "✅ Copied: fake_B=$cB, fake_A=$cA"
echo "Sample from fake_B folder:"
ls -lh "$OUT_FAKE_B" | head -n 10 || true
echo "Sample from fake_A folder:"
ls -lh "$OUT_FAKE_A" | head -n 10 || true

FID metric evaluation:

In [ ]:
%%bash
set -e
pip -q install pytorch-fid

rm -rf "$OUT_FAKE_B" "$OUT_FAKE_A"
mkdir -p "$OUT_FAKE_B" "$OUT_FAKE_A"

find "$GEN_DIR" -maxdepth 1 -type f -name "*_fake_B.png" -exec cp {} "$OUT_FAKE_B/" \;
find "$GEN_DIR" -maxdepth 1 -type f -name "*_fake_A.png" -exec cp {} "$OUT_FAKE_A/" \;

echo "FID A->B (real B vs fake_B):"
pytorch-fid "$REAL_B" "$OUT_FAKE_B" | tee ${BASE}/FID_AtoB.txt

echo "FID B->A (real A vs fake_A):"
pytorch-fid "$REAL_A" "$OUT_FAKE_A" | tee ${BASE}/FID_BtoA.txt

KID metric evaluation:

In [ ]:
%%bash
set -e
pip -q install torch-fidelity >/dev/null

# Export variables so Python can access them via os.environ
export REAL_A
export REAL_B
export OUT_FAKE_B
export OUT_FAKE_A

# 2) Run Python (no quotes needed in Heredoc since variables were already exported)
python - <<PY
import os, glob
from torch_fidelity import calculate_metrics

def count_imgs(d):
    # Recursive search + support for uppercase extensions
    exts = ("**/*.png","**/*.jpg","**/*.jpeg","**/*.PNG","**/*.JPG","**/*.JPEG")
    return sum(len(glob.glob(os.path.join(d, e), recursive=True)) for e in exts)

def extract_kid_fields(m):
    for mean_key, std_key in [
        ("kid_mean", "kid_std"),
        ("kernel_inception_distance_mean", "kernel_inception_distance_std"),
    ]:
        if mean_key in m:
            mean = float(m[mean_key])
            std  = float(m.get(std_key, float('nan')))
            return mean, std
    raise KeyError(f"KID not found in keys: {list(m.keys())}")

def run_kid(gen_dir, real_dir, label):
    n_gen  = count_imgs(gen_dir)
    n_real = count_imgs(real_dir)
    print(f"[dbg] {label}: gen={n_gen}, real={n_real}, gen_dir={gen_dir}, real_dir={real_dir}")
    if n_gen == 0 or n_real == 0:
        print(f"{label}: ❌ No samples (gen={n_gen}, real={n_real})")
        return
    subset = min(n_gen, n_real)  
        input1=gen_dir, input2=real_dir,
        cuda=True, isc=False, fid=False, kid=True,
        kid_subset_size=subset,
        kid_subset_num=100
    )
    mean, std = extract_kid_fields(m)
    print(f"{label}: KID_mean={mean:.6f} ± {std:.6f} (lower=better) | subset_size={subset}, gen={n_gen}, real={n_real}")

REAL_A = os.environ["REAL_A"]; REAL_B = os.environ["REAL_B"]
FAKE_A = os.environ["FAKE_A"]; FAKE_B = os.environ["FAKE_B"]

run_kid(FAKE_B, REAL_B, "KID A->B (fake_B vs real_B)")
run_kid(FAKE_A, REAL_A, "KID B->A (fake_A vs real_A)")
PY

LPIPS, PSNR, SSIM metric evaluations - preperations:

In [ ]:
%%bash
set -e
pip -q install pillow numpy tqdm scikit-image lpips

cat > ${BASE}/paired_metrics_from_cyclegan.py << 'PY'
import os, glob, csv
from PIL import Image
import numpy as np
from tqdm import tqdm
import torch
import lpips
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

EXSTS = (".png",".jpg",".jpeg",".bmp",".webp")

def list_images(d):
    return sorted([p for p in glob.glob(os.path.join(d, "*")) if p.lower().endswith(EXSTS)])

def stem_from_fake(path, suffix):
    base = os.path.basename(path)
    if base.endswith(suffix):
        return base[: -len(suffix)]
    if "_fake_" in base:
        return base.split("_fake_")[0]
    return os.path.splitext(base)[0]

def find_real(real_dir, stem):
    for ext in EXSTS:
        cand = os.path.join(real_dir, stem + ext)
        if os.path.isfile(cand):
            return cand
    return None

def load_rgb(path, size=None):
    im = Image.open(path).convert("RGB")
    if size is not None:
        im = im.resize(size, Image.BICUBIC)
    return np.array(im)

def to_torch_img255(arr_uint8):
    ten = torch.from_numpy(arr_uint8).permute(2,0,1).float()/255.0
    ten = ten*2.0 - 1.0
    return ten.unsqueeze(0)

def evaluate_pair_dir(real_dir, fake_dir, direction_tag, fake_suffix):
    fakes = list_images(fake_dir)
    if len(fakes)==0:
            raise SystemExit(f"No images found in {fake_dir}")
    loss_fn = lpips.LPIPS(net='alex').eval()
    if torch.cuda.is_available():
        loss_fn = loss_fn.cuda()

    rows = []
    psnr_vals, ssim_vals, lpips_vals = [], [], []

    for fp in tqdm(fakes, desc=f"{direction_tag}"):
        stem = stem_from_fake(fp, fake_suffix)
        rp = find_real(real_dir, stem)
        if rp is None:
            continue

        real = load_rgb(rp)
        fake = load_rgb(fp, size=(real.shape[1], real.shape[0]))

        psnr_v = psnr(real, fake, data_range=255)
        ssim_v = ssim(real, fake, channel_axis=2, data_range=255)

        r_t = to_torch_img255(real)
        f_t = to_torch_img255(fake)
        if torch.cuda.is_available():
            r_t = r_t.cuda(); f_t = f_t.cuda()
        lpips_v = loss_fn.forward(r_t, f_t).item()

        rows.append([os.path.basename(fp), os.path.basename(rp), psnr_v, ssim_v, lpips_v])
        psnr_vals.append(psnr_v); ssim_vals.append(ssim_v); lpips_vals.append(lpips_v)

    if len(rows)==0:
        raise SystemExit("No matching pairs found (check file names).")

    # Writing CSV file
    out_csv = f"${BASE}/{direction_tag}_pair_metrics.csv"
    with open(out_csv, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["fake_file","real_file","PSNR","SSIM","LPIPS_alex"])
        w.writerows(rows)

    import numpy as np
    print(f"{direction_tag}  Num pairs: {len(rows)}")
    print(f"PSNR  mean={np.mean(psnr_vals):.4f}  std={np.std(psnr_vals):.4f}")
    print(f"SSIM  mean={np.mean(ssim_vals):.4f}  std={np.std(ssim_vals):.4f}")
    print(f"LPIPS mean={np.mean(lpips_vals):.4f}  std={np.std(lpips_vals):.4f}")
    print(f"Saved per-image CSV -> {out_csv}")

if __name__ == "__main__":
    import argparse
    ap = argparse.ArgumentParser()
    ap.add_argument("--real_a", required=True)
    ap.add_argument("--real_b", required=True)
    ap.add_argument("--fake_a_dir", required=True)
    ap.add_argument("--fake_b_dir", required=True)
    args = ap.parse_args()

    # A→B: compare fake_B with real_B
    evaluate_pair_dir(args.real_b, args.fake_b_dir, "AtoB", "_fake_B.png")

    # B→A: compare fake_A with real_A
    evaluate_pair_dir(args.real_a, args.fake_a_dir, "BtoA", "_fake_A.png")
PY

Actual execution:

In [ ]:
%%bash
set -e

python ${BASE}/paired_metrics_from_cyclegan.py \
  --real_a "$REAL_A" \
  --real_b "$REAL_B" \
  --fake_a_dir "$OUT_FAKE_A" \
  --fake_b_dir "$OUT_FAKE_B" \
  | tee ${BASE}/PAIR_METRICS.txt

**Train a binary classifier to check accuracy:**

Build a unified dataset (JPEG 256×256) from the folders you provided:

In [ ]:
# !pip -q install pillow
import os, glob, shutil
from pathlib import Path
from PIL import Image

( ROOT/"real").mkdir(parents=True, exist_ok=True)
( ROOT/"fake").mkdir(parents=True, exist_ok=True)

def export_folder(src_dir, dst_dir, max_side=256, jpeg_q=95):
    exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.webp")
    counter = 0
    for ext in exts:
        for p in sorted(glob.glob(os.path.join(src_dir, ext))):
            try:
                img = Image.open(p).convert("RGB")
                img = img.resize((max_side, max_side), Image.BICUBIC)
                out = Path(dst_dir)/f"{Path(src_dir).name}_{Path(p).stem}_{counter}.jpg"
                img.save(out, format="JPEG", quality=jpeg_q, optimize=True)
                counter += 1
            except Exception as e:
                pass
    return counter

n_real = export_folder(TEST_A, ROOT/"real") + export_folder(TEST_B, ROOT/"real")
n_fake = export_folder(G_A, ROOT/"fake")   + export_folder(F_B, ROOT/"fake")
print("Exported -> real:", n_real, " fake:", n_fake, " to:", ROOT)

Training:

In [ ]:
import io, random, math, time
from pathlib import Path

import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models

# Config
IMG_SIZE = 256
CROP = 224
BATCH = 32
EPOCHS = 8
VAL_PCT = 0.15
TEST_PCT = 0.15
LR = 1e-4
SEED = 42
FREEZE_BACKBONE_EPOCHS = 0   

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Augmentation including random JPEG recompress
class RandomJPEG:
    def __init__(self, qmin=80, qmax=95, p=0.7):
        self.qmin, self.qmax, self.p = qmin, qmax, p
    def __call__(self, img: Image.Image):
        if random.random() > self.p:
            return img
        buf = io.BytesIO()
        img.save(buf, format="JPEG", quality=random.randint(self.qmin, self.qmax))
        buf.seek(0)
        return Image.open(buf).convert("RGB")

train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    RandomJPEG(p=0.8),
    transforms.RandomResizedCrop(CROP, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

eval_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.CenterCrop(CROP),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# Data
full_train_tf = train_tf
full_eval_tf  = eval_tf

full_ds_for_split = datasets.ImageFolder(ROOT.as_posix(), transform=full_train_tf)
idxs = np.arange(len(full_ds_for_split))
np.random.shuffle(idxs)

n = len(idxs)
n_test = int(TEST_PCT * n)
n_val  = int(VAL_PCT  * n)
n_train= n - n_val - n_test
train_idx = idxs[:n_train]
val_idx   = idxs[n_train:n_train+n_val]
test_idx  = idxs[n_train+n_val:]

# Important: validation/test transforms without augmentation
train_ds = Subset(datasets.ImageFolder(ROOT.as_posix(), transform=full_train_tf), train_idx)
val_ds   = Subset(datasets.ImageFolder(ROOT.as_posix(), transform=full_eval_tf),  val_idx)
test_ds  = Subset(datasets.ImageFolder(ROOT.as_posix(), transform=full_eval_tf),  test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=2, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

print(f"Split sizes -> train: {len(train_ds)} | val: {len(val_ds)} | test: {len(test_ds)}")

# Model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type == "cuda"))

# Optional: freeze the backbone for the first few epochs
def set_backbone_trainable(is_trainable: bool):
    for name, p in model.named_parameters():
        if not name.startswith("fc."):
            p.requires_grad = is_trainable

if FREEZE_BACKBONE_EPOCHS > 0:
    set_backbone_trainable(False)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)

best_val_acc = 0.0
best_epoch = -1

# Training Loop
for ep in range(1, EPOCHS + 1):
    t0 = time.time()
    model.train()
    run_loss, seen = 0.0, 0

    if ep == FREEZE_BACKBONE_EPOCHS + 1 and FREEZE_BACKBONE_EPOCHS > 0:
        set_backbone_trainable(True)
        optimizer = optim.Adam(model.parameters(), lr=LR)

    for xb, yb in train_dl:
        xb = xb.to(device, non_blocking=True)
        yb = yb.float().unsqueeze(1).to(device, non_blocking=True)  # Shape: [B,1]

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            logits = model(xb)
            loss = criterion(logits, yb)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        bs = xb.size(0)
        run_loss += loss.item() * bs
        seen += bs

    train_loss = run_loss / max(seen, 1)

    # Validation
    model.eval()
    probs, targs = [], []
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for xb, yb in val_dl:
            xb = xb.to(device, non_blocking=True)
            logits = model(xb).squeeze(1).cpu().numpy()
            p = 1 / (1 + np.exp(-logits))  # sigmoid
            probs.append(p)
            targs.append(yb.numpy())
    probs = np.concatenate(probs) if probs else np.array([])
    targs = np.concatenate(targs) if targs else np.array([])
    preds = (probs >= 0.5).astype(int)
    val_acc = accuracy_score(targs, preds) if targs.size else float("nan")

    dt = time.time() - t0
    print(f"Epoch {ep}/{EPOCHS} | train_loss={train_loss:.4f} | val_acc={val_acc:.3f} | {dt:.1f}s")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = ep
        torch.save(model.state_dict(), BEST_PATH)
        print("** saved best model **")

print(f"Best val acc: {best_val_acc:.3f} (epoch {best_epoch}) | path: {BEST_PATH}")

# Evaluation on the test set 
# Load the best model before evaluation
state = torch.load(BEST_PATH, map_location=device)
model.load_state_dict(state)
model.eval()

probs, targs = [], []
with torch.no_grad():
    for xb, yb in test_dl:
        xb = xb.to(device, non_blocking=True)
        logits = model(xb).squeeze(1).cpu().numpy()
        p = 1 / (1 + np.exp(-logits))
        probs.append(p)
        targs.append(yb.numpy())

probs = np.concatenate(probs) if probs else np.array([])
targs = np.concatenate(targs) if targs else np.array([])
preds = (probs >= 0.5).astype(int)

acc = accuracy_score(targs, preds) if targs.size else float("nan")
cm = confusion_matrix(targs, preds) if targs.size else None
print("\n=== Test Metrics ===")
print("Accuracy:", round(acc, 4))
if cm is not None:
    print("Confusion matrix:\n", cm)
    print("\nClassification report:\n", classification_report(targs, preds, digits=4))

Evaluate on test + save FP/FN samples + short Grad-CAM:

In [ ]:
import torch, numpy as np, matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torchvision import transforms
from torchvision.utils import save_image
from pathlib import Path

model.load_state_dict(torch.load("${BASE}/best_resnet18_real_vs_fake.pt", map_location=device))
model.eval()

all_probs, all_targs, all_imgs, all_paths = [], [], [], []
eval_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.CenterCrop(CROP),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
raw_tf  = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                              transforms.CenterCrop(CROP),
                              transforms.ToTensor()])

class ImageFolderWithPaths(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = self.loader(path)
        input_img = eval_tf(img)
        raw_img   = raw_tf(img)
        return input_img, target, raw_img, path

test_ds_paths = ImageFolderWithPaths(ROOT.as_posix())
test_ds_paths.samples = [test_ds.dataset.samples[i] for i in test_ds.indices]  

test_dl_paths = DataLoader(test_ds_paths, batch_size=16, shuffle=False, num_workers=2)

with torch.no_grad():
    for xb, yb, rawb, paths in test_dl_paths:
        xb = xb.to(device)
        p = model(xb).squeeze(1).cpu().numpy()
        all_probs.append(p); all_targs.append(yb.numpy()); all_imgs.append(rawb); all_paths += list(paths)

all_probs = np.concatenate(all_probs)
all_targs = np.concatenate(all_targs)
all_preds = (all_probs>=0.5).astype(int)
acc = accuracy_score(all_targs, all_preds)
cm  = confusion_matrix(all_targs, all_preds, labels=[0,1])
print(f"\nTest Accuracy: {acc:.3f}")
print("Confusion Matrix [rows=true 0/1, cols=pred 0/1]:\n", cm)
print("\nReport:\n", classification_report(all_targs, all_preds, target_names=['fake','real']))

Save 10 FP/FN examples to understand model behavior
out_dir = Path("${BASE}/misclassified_examples")
(out_dir/"FP").mkdir(parents=True, exist_ok=True)  # true=fake, pred=real
(out_dir/"FN").mkdir(parents=True, exist_ok=True)  # true=real, pred=fake

imgs = torch.cat(all_imgs, dim=0)
def denorm(x):
    return x.clamp(0,1)

fp_idx = np.where((all_targs==0)&(all_preds==1))[0][:10]
fn_idx = np.where((all_targs==1)&(all_preds==0))[0][:10]
for i in fp_idx:
    save_image(denorm(imgs[i]), out_dir/"FP"/f"fp_{i}.png")
for i in fn_idx:
    save_image(denorm(imgs[i]), out_dir/"FN"/f"fn_{i}.png")
print("Saved FP/FN samples to:", out_dir)

# Grad-CAM on layer4
# (simple and short: generate a heatmap for one image)
import torch.nn.functional as F

target_layer = model.layer4[-1].conv2 
grads = None
acts  = None
def save_grad(module, grad_input, grad_output):
    global grads
    grads = grad_output[0]
def save_act(module, input, output):
    global acts
    acts = output

hook_f = target_layer.register_forward_hook(save_act)
hook_b = target_layer.register_backward_hook(lambda m, gin, gout: save_grad(m, gin, gout))

# Run Grad-CAM on the first 8 test images
gc_dir = Path("${BASE}/gradcam")
gc_dir.mkdir(parents=True, exist_ok=True)

for idx in range(min(8, len(test_ds_paths))):
    x, y, raw, path = test_ds_paths[idx]
    x = x.unsqueeze(0).to(device)
    model.zero_grad()
    out = model(x)             # sigmoid
    score = out[0,0]
    score.backward()           # d score / d features
    g = grads.cpu().numpy()[0] # (C,H,W)
    a = acts.detach().cpu().numpy()[0]
    weights = g.mean(axis=(1,2))           
    cam = np.maximum((weights[:,None,None]*a).sum(axis=0), 0)
    cam = cam / (cam.max()+1e-8)
    # Save as a simple heatmap image (no fancy colors to keep code short)
    import cv2, numpy as np
    cam_img = (cam*255).astype(np.uint8)
    cam_img = cv2.resize(cam_img, (raw.shape[2], raw.shape[1]))
    cv2.imwrite(str(gc_dir/f"cam_{idx}.png"), cam_img)

hook_f.remove(); hook_b.remove()
print("Saved Grad-CAM maps to:", gc_dir)

Display the Grad-CAM images:

In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

FOLDER = Path("${BASE}/gradcam")  

IMG_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp", ".gif"}

paths = sorted([p for p in FOLDER.iterdir() if p.is_file() and p.suffix.lower() in IMG_EXTS])[:7]

if not paths:
    print("No images found in folder:", FOLDER)
else:
    n = len(paths)
    if n >= 5:
        rows, cols = 2, 4
    else:
        rows, cols = 1, n

    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    axes = axes.ravel() if isinstance(axes, (list, tuple)) or hasattr(axes, "ravel") else [axes]

    for i, p in enumerate(paths):
        axes[i].imshow(Image.open(p))
        axes[i].set_title(p.name, fontsize=8)
        axes[i].axis("off")

    for j in range(len(paths), rows*cols):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()

Display sample images:

In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

FOLDER = Path("${BASE}/misclassified_examples/FP")  

IMG_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp", ".gif"}

paths = sorted([p for p in FOLDER.iterdir() if p.is_file() and p.suffix.lower() in IMG_EXTS])[:8]

if not paths:
    print("No images found in folder:", FOLDER)
else:
    n = len(paths)
    if n >= 5:
        rows, cols = 2, 4
    else:
        rows, cols = 1, n

    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    axes = axes.ravel() if isinstance(axes, (list, tuple)) or hasattr(axes, "ravel") else [axes]

    for i, p in enumerate(paths):
        axes[i].imshow(Image.open(p))
        axes[i].set_title(p.name, fontsize=8)
        axes[i].axis("off")

    for j in range(len(paths), rows*cols):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()